In [34]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")
import pickle

%reload_ext autoreload
%autoreload 2
from config import keys
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns


# University of Copenhagen Dataset

In [35]:
df_1 = pd.read_csv('data/NAACL_SRW_2016.csv')
df_1.head()

,572342978255048705,racism
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism


In [36]:
df_1.columns = ['id', 'label']
df_1.label.value_counts()

none      11559
sexism     3378
racism     1969
Name: label, dtype: int64

In [37]:
df_racsex = df_1[(df_1['label']=='racism') | (df_1['label']=='sexism')]

In [38]:
def group_list(lst, size=100):
    new_list =[]
    idx = 0
    while idx < len(lst):        
        new_list.append(','.join([str(item) for item in lst[idx:idx+size]]))
        idx += size
    return new_list

def tweets_request(tweets_ids):
    lst = []
    for batch in tqdm(tweets_ids):
        url = "https://api.twitter.com/2/tweets?ids={}&&tweet.fields=created_at,entities,geo,id,public_metrics,text&user.fields=description,entities,id,location,name,public_metrics,username".format(batch)
        payload={}
        headers = {'Authorization': 'Bearer ' + keys['bearer_token'],
        'Cookie': 'personalization_id="v1_hzpv7qXpjB6CteyAHDWYQQ=="; guest_id=v1%3A161498381400435837'}
        r = requests.request("GET", url, headers=headers, data=payload)
        data = r.json()
        if 'data' in data.keys():
            lst.append(pd.DataFrame(data['data']))
    return pd.concat(lst)


In [39]:
racsex_id = group_list(list(df_racsex.id))

In [40]:
df_rac_sex = tweets_request(racsex_id)

100%|██████████| 54/54 [00:29<00:00,  1.81it/s]


In [41]:
df_rac_sex['label'] = 1
df_1= df_rac_sex[['text', 'label']]
df_1.head()

,text,label
0,Drasko they didn't cook half a bird you idiot ...,1
1,Hopefully someone cooks Drasko in the next ep ...,1
2,of course you were born in serbia...you're as ...,1
3,These girls are the equivalent of the irritati...,1
0,RT @YesYoureRacist: At least you're only a tin...,1


In [42]:
df_1.count()

text     2664
label    2664
dtype: int64

# Davidson Dataset

In [43]:
df_2 = pd.read_csv("data/labeled_data.csv", index_col=0)
df_2.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [44]:
df_2 = df_2.drop(columns=['count', 'hate_speech', 'offensive_language', 'neither'], axis=1)
df_2.columns = ['label', 'text']
df_2.label.value_counts()

1    19190
2     4163
0     1430
Name: label, dtype: int64

In [45]:
df_2 = df_2.replace({'label': {1: 0, 2: 0, 0: 1}})
df_2.label.value_counts()

0    23353
1     1430
Name: label, dtype: int64

In [46]:
df_2.head()

,label,text
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [47]:
df_2.count()

label    24783
text     24783
dtype: int64

# Aristotle Dataset

In [48]:
df_3 = pd.read_csv("data/hatespeechtwitter.csv")
df_3.head()

,tweet_id,maj_label
0,849667487180259329,abusive
1,850490912954351616,abusive
2,848791766853668864,abusive
3,848306464892604416,abusive
4,850010509969465344,normal


In [49]:
df_3.columns = ['id', 'label']
df_3.label.value_counts()

normal     52835
spam       13404
abusive    10122
hateful     3635
Name: label, dtype: int64

In [50]:
# setup url and headers for calling Twitter API as per documentation
url = "https://api.twitter.com/2/tweets?ids=847661947159891972,847799130277675008,848933211375779840&tweet.fields=created_at,entities,geo,id,public_metrics,text&user.fields=description,entities,id,location,name,public_metrics,username"
payload={}
headers = {'Authorization': 'Bearer ' + keys['bearer_token'], 'Cookie': 'personalization_id="v1_hzpv7qXpjB6CteyAHDWYQQ=="; guest_id=v1%3A161498381400435837'}

# make GET request to Twitter API to get response object 
r = requests.request("GET", url, headers=headers, data=payload)

# convert payload to dictionary
data = r.json()

In [51]:
df_hateful = df_3[df_3['label']=='hateful']
hate_ids = group_list(list(df_hateful.id))

In [52]:
df_hate = tweets_request(hate_ids)

100%|██████████| 37/37 [00:23<00:00,  1.59it/s]


In [53]:
df_hate['label'] = 1
df_3 = df_hate[['text', 'label']]
df_3.head()

,text,label
0,I hate er chase because if the Bitch that work...,1
1,RT @nyctophil3: Pineapples do not belong on pi...,1
2,Niggas keep talking about women wearing weave ...,1
3,@vappywave idiot that's not gonna work. you go...,1
4,RT @ayevonnn: bruh i fucking hate people like ...,1


In [54]:
df_3.count()

text     1607
label    1607
dtype: int64

# HASOC Dataset

In [55]:
df_4 = pd.read_csv("data/english_dataset.tsv", delimiter = "\t")
df_4.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE


In [56]:
df_4 = df_4[df_4.task_2 == 'HATE']
df_4['label'] = 1
df_4 = df_4[['text', 'label']]
df_4.head()

,text,label
1,@politico No. We should remember very clearly ...,1
11,I don’t know how much more I can take! 45 is a...,1
23,#ShameOnICC 1. ICC on Dhoni's gloves ...,1
27,Disgusting! And they don’t change their cultur...,1
34,"@TheRealOJ32 Of all the retired NFL players, w...",1


In [57]:
df_4.count()

text     1143
label    1143
dtype: int64

# Combined Dataframe

In [58]:
combined = pd.concat([df_1, df_2, df_3, df_4], ignore_index=True)
combined.label.value_counts()

0    23353
1     6844
Name: label, dtype: int64

In [59]:
combined.head()

,text,label
0,Drasko they didn't cook half a bird you idiot ...,1
1,Hopefully someone cooks Drasko in the next ep ...,1
2,of course you were born in serbia...you're as ...,1
3,These girls are the equivalent of the irritati...,1
4,RT @YesYoureRacist: At least you're only a tin...,1


In [60]:
combined.count()

text     30197
label    30197
dtype: int64

In [61]:
combined.to_csv("data/combination.csv")